Universidad del Valle de Guatemala

## Proyecto: Data Science

Andrea Amaya 19357 | Brandon Hernández	 19376 |		Martin Amado	19020 |   Juan Pablo Pineda 19087

In [ ]:
# run once
import glob, os
os.chdir('./cvs_data')

In [ ]:
import pandas as pd 
import re
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None 

In [ ]:
data_frames = []
for file in glob.glob('*.csv'):
  data_frames.append(pd.read_csv(file, encoding='utf-8'))
data = pd.concat(data_frames, ignore_index=True)
data = data.apply(lambda x: x.astype(str).str.lower()) # Todo a minusculas

In [ ]:
data

#### Limpiando los numeros de telefono

In [ ]:
def cleanNumbers(number):
    countingNumbers = 0
    if len(number) == 8 and number.isdigit():
        return [number]
    for possibleNumber in number:
        if not possibleNumber.isdigit():
            # Si el delimitador es un - y habian 4 digitos antes
            if (possibleNumber == '-' and countingNumbers <= 4):
                # Se remueve el -
                number = number.replace('-', '', 1) # Se reemplaza el primer -
                countingNumbers = 0
            else:
                # Si no es un numero, se intenta separar los digitos por ese delimitador
                separatedNumbers = number.split(possibleNumber)
                separatedNumbers = [sep.replace(" ", "") for sep in separatedNumbers] # Removiendo posibles espacios
                separatedNumbers = [sep.replace("-", "") for sep in separatedNumbers] # Removiendo posibles guiones


                correctNumbers = []
                # Por cada numero
                for newNumber in separatedNumbers:
                    # Si la longitud es de 8 caracteres, es un numero correcto
                    if len(newNumber) == 8:
                        correctNumbers.append(newNumber)
                    # Si la longitud es de 2, es posible que sean varios numeros
                    elif len(newNumber) == 2 and len(correctNumbers) >= 1:
                        lastAddedNumber = correctNumbers[len(correctNumbers) - 1]
                        twoLastDigits = int(lastAddedNumber[-2:])
                        newNumber = int(newNumber)
                        # Si el delimitador es un /
                        if possibleNumber == '/':
                            correctNumbers.append(lastAddedNumber[:6] + '' + str(newNumber))
                        # Si el delimitador es un -
                        elif possibleNumber == '-':
                            while (twoLastDigits < newNumber):
                                twoLastDigits += 1
                                correctNumbers.append(lastAddedNumber[:6] + '' + str(twoLastDigits))
                
                if (len(correctNumbers) == 0 and number != 'nan'):
                    print('OOPS! Se retorna [], el valor recibido fue: ', number)
                return correctNumbers
        else:
            countingNumbers += 1

In [ ]:
for index, row in data.iterrows():
    row["TELEFONO"] = cleanNumbers(row["TELEFONO"])

# Se crea n rows como n numeros hayan
data = data.explode('TELEFONO')

In [ ]:
data.shape[0]

#### Limpieza municipios (revision de que no hayan repetidos)

In [ ]:
data['MUNICIPIO'].unique()

#### Limpieza (revision de que no hayan repetidos)

<style>
td {
  font-size: 12px
}
</style>
### 
|VARIABLE|VALORES|
| --- | --- |
|DEPARTAMENTO|alta verapaz |
|NIVEL|diversificado |
|SECTOR|'privado', 'oficial', 'municipal', 'cooperativa' |
|AREA|'urbana', 'rural', 'sin especificar'|
|STATUS|'abierta', 'cerrada temporalmente', 'temporal nombramiento', 'temporal titulos'|
|MODALIDAD|'monolingue', 'bilingue'|
|JORNADA|'matutina', 'vespertina', 'doble', 'nocturna', 'sin jornada', 'intermedia'|
|PLAN|'diario(regular)', 'fin de semana', 'a distancia', 'semipresencial', 'virtual a distancia', 'sabatino', 'semipresencial (fin de semana)', 'dominical', 'mixto', |
|DEPARTAMENTAL|alta verapaz|
### 


#### Limpieza departamentos (revision de que no hayan repetidos)

In [ ]:
# Si cuenta con los 23 departamentos
data['DEPARTAMENTO'].unique()

In [ ]:
# Se cuenta con el nivel diversificado
data['NIVEL'].unique()

In [ ]:
# Se cuenta con los sectores especificos
data['SECTOR'].unique()

In [ ]:
# Se cuenta con las areas especificas
data['AREA'].unique()

In [ ]:
# Se cuenta con los estatus especificos
data['STATUS'].unique()

In [ ]:
# Se cuenta con las modalidades especificas
data['MODALIDAD'].unique()

In [ ]:
# Se cuenta con las jornadas especificas
data['JORNADA'].unique()

In [ ]:
data['DEPARTAMENTAL'].unique()

### Datos a limpiar en la columna direccion
Aparecen los siguientes errores:
    * avaenida
    * av. sera cambiado por avenida
    * 3ra. sera cambiado por 3a.
    * 1a sera cambiado por 1a.
    * Comillas simples seran cambiadas a comillas dobles
    * 11avenida
    * 1a, sera cambiado por 1a.
    * 5ta / 5ta. sera cambiado por 3a.
    

LIMPIEZA DE AVENIDAS

In [ ]:
def changeAvenueShort(avenue):
    try:
        splitAvenue = avenue.split()
        for i in range(len(splitAvenue)):
                word = splitAvenue[i]
                if ("av" in word):
                    # ej. ave.1, av1
                    match1 = re.match(r"([a-z]+)(.*?)([0-9]+)", word, re.I)
                    # ej.  2avenida, 2avenida
                    match2 = re.match(r"([0-9]+)([a-z]+)", word, re.I)
                    if match1:
                        items = list(match1.groups())
                        str_list = list(filter(None, items)) # Remove empty spaces
                        '''
                        ['avenida', '1']
                        ['av', '.-', '1']
                        ['avenida', '4']
                        ['av', '.', '15']
                        '''
                        str_list[0] = 'av.'
                        splitAvenue[i] = str_list[0] + ' ' + str_list[len(str_list) - 1]
                    if match2 and '.avenida' not in word and len(word) < 11 and word != '5a.av.zona' and word != '2a.av.':
                        items = list(match2.groups())
                        str_list = list(filter(None, items)) # Remove empty spaces
                        '''
                        ['11', 'avenida']
                        ['3', 'avenida']    
                        ['4', 'av']
                        ['5', 'a']
                        ['6', 'av']
                        '''
                        str_list[1] = 'av.'
                        splitAvenue[i] = str_list[0] + ' ' + str_list[1]
                    elif match2:
                        '''
                        6a.avenida
                        5a.avenida
                        6a.avenida
                        9na.avenida
                        1a.avenida
                        1a.avenida
                        1ra.ave.7-46
                        1ra.ave.7-46
                        1ra.ave.7-46
                        1ra.av.2da.calle
                        3ra.av.5-60
                        3ra.av.5-60
                        10calle3ra.ave.
                        5ta..avenida,
                        5a.av.zona
                        5a.av.zona
                        10m.av.zona
                        1ra.avenida
                        1ra.avenida
                        1ra.avenida
                        1ra.avenida
                        2a.av.
                        '''
                        split2 = word.split('.')
                        for i in range(len(split2)):
                            if "av" in split2[i]:
                                split2[i] = "av."

                        
                        splitAvenue[i] = ' '.join(split2)
                    if not match1 and not match2:
                        if (word[0] == 'a' and word != 'aviacion'):
                            splitAvenue[i] = 'av.'
                        elif (word == '.av.sur'):
                            splitAvenue[i] = 'av. sur.'
                        else:
                            '''
                            # Ignore: lavarreda, aviacion,  claveles, cañaveral, bravo, maravillas, pavón
                            # Ignore: navarra, pavon,  javier, primavera, brava, javia, navidad
                            # Ignore: pavarrotti, xecanchavox
                            '''
        return ' '.join(splitAvenue)
    except:
        '''
        nan
        '''
        return avenue


In [ ]:
data = data.replace('nan', np.nan)
# pd.set_option("display.max_rows", None, "display.max_columns", None)

for index, row in data.iterrows():
    row["DIRECCION"] = changeAvenueShort(row["DIRECCION"])


data['DIRECCION']


### Datos a limpiar en la columna supervisor
Aparecen los siguientes errores:
    * se elimino el distrito
    * vacante vacante
    * 'nan'
    * 'juan carlos morales gonzalez', 'juan carlos morales gonzález',
    * 'augusto manuel andres sebastian',  'augusto manuel andrés sebastián',


In [ ]:
data['SUPERVISOR'] = data['SUPERVISOR'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
data = data.replace('vacante vacante', np.nan)
data = data[data['SUPERVISOR'] != 'se elimino el distrito'] # Si se eliminó, no se necesita la data

In [ ]:
data['SUPERVISOR'].unique()